In [1]:
# This Python 3 environment comes with ma
# ny helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# TaskFlow: An Autonomous Research & Execution Agent 🧠⚙️

This project demonstrates a GenAI-based autonomous agent that:
- Understands complex research queries
- Breaks them into subtasks
- Retrieves and summarizes relevant info using LLM
- Outputs structured research reports (in JSON/Markdown)

Models used: `gemini-1.5-pro`, `text-embedding-004`


In [3]:
from google import genai
from google.genai import types
import json

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

ModuleNotFoundError: No module named 'kaggle_secrets'

In [ ]:
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
for model in client.models.list():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
m

### Using the gemini-1.5-pro, and answering a prompt 

In [ ]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=2000)


response = client.models.generate_content(
    model = 'gemini-1.5-pro',
    config = short_config,
    contents = 'Find best open-source LLMs with license info in 200 words'
)

print(response.text)

Several excellent open-source LLMs offer varying capabilities and permissive licenses.

**Llama 2** from Meta, under a custom license allowing commercial use but restricting large-scale deployments, provides strong performance across various benchmarks.

**Falcon** by TII, licensed under Apache 2.0, offers competitive performance with excellent instruction-following abilities.

**MPT** from MosaicML, also Apache 2.0, is designed for commercial usability and efficient inference.

**Dolly 2.0** from Databricks, also under Apache 2.0,  focuses on instruction following and is fine-tuned for chat applications.

**BLOOM** from BigScience, licensed under RAIL License 1.0, a responsible AI license, is multilingual and emphasizes ethical considerations.

Choosing the "best" depends on specific needs, considering license implications, performance characteristics, and downstream tasks.  Researching benchmarks and community feedback is crucial for informed selection.



### Responding some user queries and returning the response in json format

In [ ]:
import google.generativeai as genai
import typing_extensions as typing
import json
from google.genai import types


genai.configure(api_key= GOOGLE_API_KEY)


class TaskPlan(typing.TypedDict):
    tasks: list[str]

config = types.GenerateContentConfig(
    temperature=0.2,
    response_mime_type="application/json",
    response_schema=TaskPlan,
)

def plan_tasks(topic: str):
    response = client.models.generate_content(
        model ='gemini-1.5-pro',
        config=config,
        contents=f"Break into 3–5 research tasks: '{topic}'. Return JSON with key 'tasks'."
    )
    return response.text

user_query = "Role of AI in sustainable development" # queries
tasks = plan_tasks(user_query)


parsed_tasks = json.loads(tasks)
print(json.dumps(parsed_tasks, indent=2)) #for proper format

{
  "tasks": [
    "1. Analyze existing AI applications for sustainable development across various sectors (e.g., energy, agriculture, transportation). Identify successful implementations, challenges, and opportunities for improvement.",
    "2. Investigate the ethical implications of using AI for sustainable development, including bias in algorithms, data privacy, and potential job displacement. Develop guidelines for responsible AI development and deployment.",
    "3. Evaluate the role of AI in promoting circular economy principles, such as waste reduction, resource optimization, and product lifecycle management. Explore AI-powered solutions for sustainable consumption and production.",
    "4. Assess the potential of AI to enhance climate change mitigation and adaptation strategies. Analyze how AI can improve climate modeling, prediction, and disaster response.",
    "5. Develop a framework for integrating AI into national and international sustainable development policies. Identif

# Agent Loop
[ User query ] → [ Task planner ] → [ Subtasks ] → [ Execute each task ]

- It decomposes each tasks into subtasks
- Then executes each subtasks using GenAI


In [ ]:
import google.generativeai as genai
model = genai.GenerativeModel(model_name="gemini-1.5-flash") # using a different model as gemini 1.5 pro has limitations on rpm



for i, task in enumerate(parsed_tasks['tasks'], 1):
    print(f"\n🔹 Task {i}: {task}")
    
    # Ask Gemini to answer each task
    response = model.generate_content(f"Write a short paragraph on: {task}")
    
    print("📝 Response:")
    print(response.text)



🔹 Task 1: 1. Analyze existing AI applications for sustainable development across various sectors (e.g., energy, agriculture, transportation). Identify successful implementations, challenges, and opportunities for improvement.
📝 Response:
AI applications are increasingly contributing to sustainable development across diverse sectors.  Successful implementations include precision agriculture using AI-powered drones for optimized irrigation and fertilization, leading to reduced resource consumption; smart grids leveraging AI for efficient energy distribution and renewable energy integration; and AI-driven traffic management systems optimizing transportation flows and reducing emissions.  However, challenges remain, including data scarcity and bias in training datasets, high computational costs limiting accessibility, and concerns about algorithmic transparency and accountability.  Opportunities for improvement lie in developing more robust, explainable, and ethically sound AI models, fos

# Document Understanding

Reading documents and extracting text from it

In [ ]:
!pip install pymupdf

import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf('/kaggle/input/syllabus/btech_cse_3rd_year_1624712911.pdf')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 63.1 MB/s eta 0:00:00


Breaking the long pdf into chunks so that it is made easier for the model to understand and give output efficiently with minimal errors.

In [ ]:
def chunk_text(text, chunk_size=100, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(raw_text)



### Embed the chunks

Embedding the chunks into vectors for semantic understanding.

using embedding model for embedding the chunks.

In [ ]:
# batching the chunks as the request is limited to 100
def batch_chunks(chunks, batch_size=100):
    for i in range(0, len(chunks), batch_size):
        yield chunks[i:i + batch_size]

all_embeddings = []

for batch in batch_chunks(chunks, batch_size=100):
    response = client.models.embed_content(
        model='models/text-embedding-004',
        contents=batch,
        config=types.EmbedContentConfig(task_type='semantic_similarity')
    )
    all_embeddings.extend([r.values for r in response.embeddings])



The embeddedd vectors are now being stored in chunk_db

In [ ]:
# Assuming all_embeddings and chunks are in order
chunk_db = [
    {"text": chunk, "embedding": embedding}
    for chunk, embedding in zip(chunks, all_embeddings)
]


Also embedding the query into vectors so that the model can match these embedding with the relevant content from the pdf.

In [ ]:
query = "what is the topics of soft computing and explain each topic within it"

In [ ]:
query_embedding_response = client.models.embed_content(
    model='models/text-embedding-004',
    contents=[query],
    config=types.EmbedContentConfig(task_type='semantic_similarity')
)
query_embedding = query_embedding_response.embeddings[0].values


Calculating the similarity scores using cosine similarity and finding the top k similar chunks

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

# Find top-k most relevant chunks
top_k = 3
scored_chunks = [
    (cosine_similarity(np.array(query_embedding), np.array(chunk['embedding'])), chunk['text'])
    for chunk in chunk_db
]

# Sort by score
ranked_chunks = sorted(scored_chunks, key=lambda x: x[0], reverse=True)
top_chunks = [text for _, text in ranked_chunks[:top_k]]


Giving a rag prompt that specifies what to do to the model

and generating the output using the gemini model.


In [ ]:
rag_prompt = (
    "Answer the following question using the provided context.\n\n"
    f"Context:\n{''.join(top_chunks)}\n\n"
    f"Question:\n{query}"
)

model = genai.GenerativeModel('gemini-1.5-pro')
response = model.generate_content(rag_prompt)

print("🔍 Final Answer:\n", response.text)


🔍 Final Answer:
 The context mentions three main topics within soft computing:

1. **Fuzzy Logic:**  Fuzzy logic deals with uncertainty and imprecise information. Unlike traditional binary logic (true/false), fuzzy logic allows for degrees of truth.  It uses fuzzy sets, which can have partial membership. For example, the concept of "tall" is fuzzy. Someone who is 6'2" is definitely tall, someone who is 5'2" is definitely not tall, but someone who is 5'10" might have a partial membership in the set of "tall" people. This allows for more nuanced and human-like reasoning in situations where precise definitions are difficult or impossible.

2. **Neural Networks:** Inspired by the biological nervous system, neural networks are computing systems made of interconnected nodes (neurons) organized in layers.  These networks learn from data by adjusting the strengths of the connections between neurons. They are particularly good at pattern recognition, classification, and prediction tasks, and ar